# Imports

In [1]:
!pip install -q transformers datasets
!apt-get install rar

     |████████████████████████████████| 4.4 MB 30.9 MB/s 
     |████████████████████████████████| 362 kB 22.3 MB/s 
     |████████████████████████████████| 6.6 MB 52.2 MB/s 
     |████████████████████████████████| 596 kB 69.8 MB/s 
     |████████████████████████████████| 101 kB 11.5 MB/s 
     |████████████████████████████████| 212 kB 75.5 MB/s 
     |████████████████████████████████| 1.1 MB 45.6 MB/s 
     |████████████████████████████████| 140 kB 55.0 MB/s 
     |████████████████████████████████| 127 kB 57.6 MB/s 
     |████████████████████████████████| 271 kB 47.3 MB/s 
     |████████████████████████████████| 144 kB 62.9 MB/s 
     |████████████████████████████████| 94 kB 1.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Reading package lists... Done
Building 

In [2]:
import pandas as pd
import numpy as np
import ast

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import TextVectorization
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EvalPrediction
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import torch

In [3]:
from google.colab import drive, auth
# Mount Google Drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
base_dir = './drive/MyDrive/University/00012-NLP/MovieClassification/'
# base_dir = './'

# Bert Sequence Generator

In [5]:
dataset = load_dataset('csv', data_files={'train': base_dir + 'data/cleaned/data.csv'})
dataset

Using custom data configuration default-708bb0d4e9ba2d40


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-708bb0d4e9ba2d40/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Title', 'MPA', 'Plot', 'Normalized_Plot'],
        num_rows: 27401
    })
})

In [6]:
dataset['train'][0]

{'MPA': 'G',
 'Normalized_Plot': "['lion', 'prince', 'simba', 'father', 'targeted', 'bitter', 'uncle', 'want', 'ascend', 'throne']",
 'Plot': 'Lion prince Simba and his father are targeted by his bitter uncle, who wants to ascend the throne himself.',
 'Title': 'The Lion King',
 'Unnamed: 0': 0}

In [7]:
labels = ['G', 'PG', 'PG-13', 'R']
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['G', 'PG', 'PG-13', 'R']

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [9]:
def preprocess_data(examples):   
    for i in range(len(examples['Normalized_Plot'])):
        examples['Normalized_Plot'][i] = ' '.join(ast.literal_eval(examples['Normalized_Plot'][i]))
    
    labels_batch_array = []
    # take a batch of texts
    text = examples["Normalized_Plot"]
    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    rate = examples['MPA']
    for i in range(len(text)):
        labels_batch = {'G': False, 'PG': False, 'PG-13': False, 'R': False}
        labels_batch[rate[i]] = True
        labels_batch_array.append(labels_batch)

    # print(labels_batch_array)

    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))
    # fill numpy array
    for i in range(len(text)):
        temp_dic = labels_batch_array[i]
        for idx, label in enumerate(labels):
            # print(temp_dic)
            labels_matrix[i, idx] = temp_dic[label]

    encoding["labels"] = labels_matrix.tolist()
    
    return encoding

In [10]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)
encoded_dataset['train'][0].keys()

Parameter 'function'=<function preprocess_data at 0x7f58c82f5680> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/28 [00:00<?, ?ba/s]

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [11]:
encoded_dataset.set_format("torch")

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    problem_type="multi_label_classification", 
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
    )

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [13]:
args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
)

In [14]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [15]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["train"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [16]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 27401
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 17130


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.476500,0.432192,0.411011,0.625531,0.284990
2,0.414900,0.339437,0.654537,0.754389,0.557206
3,0.317100,0.215881,0.839226,0.883891,0.804277
4,0.238300,0.141752,0.905000,0.930951,0.883544
5,0.181200,0.113787,0.925432,0.947617,0.914492


***** Running Evaluation *****
  Num examples = 27401
  Batch size = 8
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-3426
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-3426/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-3426/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-3426/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-3426/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 27401
  Batch size = 8
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-6852
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-6852/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-6852/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-6852/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/

TrainOutput(global_step=17130, training_loss=0.3347529503175966, metrics={'train_runtime': 4810.6129, 'train_samples_per_second': 28.48, 'train_steps_per_second': 3.561, 'total_flos': 9012044363013120.0, 'train_loss': 0.3347529503175966, 'epoch': 5.0})

In [17]:
pt_save_directory = base_dir + 'models/fine_tuned_bert'
tokenizer.save_pretrained(pt_save_directory)
model.save_pretrained(pt_save_directory)

tokenizer config file saved in ./drive/MyDrive/University/00012-NLP/MovieClassification/models/fine_tuned_bert/tokenizer_config.json
Special tokens file saved in ./drive/MyDrive/University/00012-NLP/MovieClassification/models/fine_tuned_bert/special_tokens_map.json
Configuration saved in ./drive/MyDrive/University/00012-NLP/MovieClassification/models/fine_tuned_bert/config.json
Model weights saved in ./drive/MyDrive/University/00012-NLP/MovieClassification/models/fine_tuned_bert/pytorch_model.bin


In [18]:
zip_path = base_dir + 'models/fine_tuned_bert.zip'

!zip -r zip_path pt_save_directory

	zip warning: name not matched: pt_save_directory

zip error: Nothing to do! (try: zip -r zip_path . -i pt_save_directory)
